# Model Evaluation 4

---

__This Notebook__

Uses a more efficient approach to evaluate classifiers, as demonstrated by Cole Brendel in his article: [Quickly Compare Multiple Models](https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0).


## Setup

In [1]:
import re
import os
import sys
import time
import joblib 

import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt

from datetime import datetime
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, \
    RandomForestClassifier, GradientBoostingClassifier

from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, \
    confusion_matrix

# custom
import custom.evaluate_models as E

# set options
np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_colwidth = 999
dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-03-06


## Load Raw Data

In [2]:
def load_raw(data):
    raw_path = os.path.join("data","1_raw")
    filename = ''.join([data, ".csv"])
    out_dfm = pd.read_csv(os.path.join(raw_path, filename))
    out_arr = np.array(out_dfm.iloc[:,0].ravel())
    return out_arr

X_train_raw = load_raw("X_train")
X_test_raw = load_raw("X_test")
y_train_array = load_raw("y_train")
y_test_array = load_raw("y_test") 

def make_int(y_array):
    y = y_array.copy()
    y[y=='ham'] = 0
    y[y=='spam'] = 1
    y = y.astype('int')
    return y

y_train = make_int(y_train_array)
y_test = make_int(y_test_array)

## Load Preprocessed Data

In [3]:
def load_X(filename):
    proc_dir = os.path.join("data", "2_processed")
    filename = ''.join([filename, '.npz'])
    X = sp.load_npz(os.path.join(proc_dir, filename))
    return X

X_train_processed = load_X('X_train_processed')
X_test_processed = load_X('X_test_processed')

## Instantiate Candidate Models

In [7]:
models = [
    (
        'KNN', KNeighborsClassifier(
        n_neighbors=3 # default (5) had bad spam recall
    )),
    (
        'XGboost', XGBClassifier(
        seed=42, eval_metric='error', # try logloss?
        use_label_encoder=False
    )),
    (
        'AdaBoost', AdaBoostClassifier(
        random_state=42 , n_estimators=10, 
        learning_rate=0.001
    )),
    (
        'RandomForest1', RandomForestClassifier(
        random_state=42, n_estimators=100, max_features=150, 
        max_depth=8, min_samples_split=3, n_jobs=1
    )),           
    (
        'RandomForest2', RandomForestClassifier(
        random_state=42, n_estimators=100, max_features=300, 
        max_depth=8, min_samples_split=3, n_jobs=1
    )),
    (
        'GBoost1a', GradientBoostingClassifier(
        random_state=42, n_estimators=50, max_features=None, 
        max_depth=1, min_samples_split=2
    )),
    (
        'Gboost2a', GradientBoostingClassifier(
        random_state=42, n_estimators=100, max_features=300, 
        max_depth=8, min_samples_split=5
    )),
    (
        'Gboost2c', GradientBoostingClassifier(
        random_state=42, n_estimators=50, max_features=300, 
        max_depth=3, min_samples_split=5
    ))    
]

Code below adapted from Cole Brendel in [Quickly Compare Multiple Models](https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0).

In [8]:
def run_exps(models: list(), 
             X_train: pd.DataFrame , 
             y_train: pd.DataFrame, 
             X_test: pd.DataFrame, 
             y_test: pd.DataFrame) -> pd.DataFrame:
    """
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    """    
    
    dfs, results, names = [], [], []
    
    scoring = ['accuracy', 'precision_weighted', 
               'recall_weighted', 'f1_weighted', 
               'roc_auc']
    
    target_names = ['ham', 'spam']
    
    for name, model in models:
        start_ = time.time()
        kfold = model_selection.KFold(n_splits=5, 
                                      shuffle=True, 
                                      random_state=42)
        
        cv_results = model_selection.cross_validate(model, 
                                                    X_train, 
                                                    y_train, 
                                                    cv=kfold, 
                                                    scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        print(name)
        print(classification_report(y_test, 
                                    y_pred,
                                    digits=3,
                                    target_names=target_names))
            
        # collect results
        results.append(cv_results)
        names.append(name)
        
        df = pd.DataFrame(cv_results)
        df['model'] = name
        dfs.append(df)
                
    final = pd.concat(dfs, ignore_index=True)
    return final

In [10]:
df_splits = run_exps(models, X_train_processed, y_train, X_test_processed, y_test)

KNN
              precision    recall  f1-score   support

         ham      0.929     1.000     0.963      1442
        spam      1.000     0.517     0.682       230

    accuracy                          0.934      1672
   macro avg      0.964     0.759     0.822      1672
weighted avg      0.938     0.934     0.924      1672

XGboost
              precision    recall  f1-score   support

         ham      0.995     0.993     0.994      1442
        spam      0.957     0.970     0.963       230

    accuracy                          0.990      1672
   macro avg      0.976     0.981     0.979      1672
weighted avg      0.990     0.990     0.990      1672

AdaBoost
              precision    recall  f1-score   support

         ham      0.994     0.993     0.994      1442
        spam      0.957     0.965     0.961       230

    accuracy                          0.989      1672
   macro avg      0.976     0.979     0.977      1672
weighted avg      0.989     0.989     0.989      1672

Funny how XGBoost performed well pretty much out of the box. 

In [11]:
df_splits

,fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted,test_roc_auc,model
0,0.047264,13.943518,0.923077,0.927776,0.923077,0.911401,0.824853,KNN
1,0.041493,13.266284,0.920513,0.927236,0.920513,0.906472,0.797741,KNN
2,0.042892,12.187585,0.923077,0.927200,0.923077,0.908714,0.810261,KNN
3,0.016919,5.203025,0.917949,0.925093,0.917949,0.902767,0.787678,KNN
4,0.015626,5.172785,0.944872,0.946737,0.944872,0.937167,0.859672,KNN
5,5.580916,0.406632,0.994872,0.994935,0.994872,0.994890,0.993751,XGboost
6,5.590989,0.405059,0.985897,0.985850,0.985897,0.985870,0.998318,XGboost
7,3.435742,0.397466,0.993590,0.993572,0.993590,0.993547,0.999718,XGboost
8,6.801255,0.461302,0.996154,0.996146,0.996154,0.996146,0.999932,XGboost
9,6.839387,0.463065,0.994872,0.994853,0.994872,0.994846,0.999619,XGboost


As expected, KNN is lightning fast, and XGBoost is the slowest, but not too much... although that's highly dependent on amount of data and use case.

In [ ]:
#bootstraps = []
#for model in list(set(final.model.values)):
#    model_df = final.loc[final.model == model]
#    bootstrap = model_df.sample(n=30, replace=True)
#    bootstraps.append(bootstrap)
#        
#bootstrap_df = pd.concat(bootstraps, ignore_index=True)
#results_long = pd.melt(bootstrap_df,id_vars=['model'],var_name='metrics', value_name='values')
#time_metrics = ['fit_time','score_time'] # fit time metrics
#
### PERFORMANCE METRICS
#results_long_nofit = results_long.loc[~results_long['metrics'].isin(time_metrics)] # get df without fit times
#results_long_nofit = results_long_nofit.sort_values(by='values')
#
### TIME METRICS
#results_long_fit = results_long.loc[results_long['metrics'].isin(time_metrics)] # df with fit times
#results_long_fit = results_long_fit.sort_values(by='values')

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#plt.figure(figsize=(20, 12))
#sns.set(font_scale=2.5)
#g = sns.boxplot(x="model", y="values", hue="metrics", data=results_long_nofit, palette="Set3")
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.title('Comparison of Model by Classification Metric')
#plt.savefig('./benchmark_models_performance.png',dpi=300)

---